**ETL 2**

In [1]:
import pandas as pd
import re

*---------------------------------------------------------------------games.csv---------------------------------------------------------------------*

1. importamos los datos de games y analizamos sus columnas y el tipo de datos que tienen

In [2]:
games = pd.read_parquet('datos/games.parquet')

In [3]:
#games.describe(include='all')

In [4]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29830 entries, 0 to 29829
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        29830 non-null  object
 1   title         29830 non-null  object
 2   id            29830 non-null  int64 
 3   release_year  29830 non-null  object
dtypes: int64(1), object(3)
memory usage: 932.3+ KB


puedo verificar que tengo datos **NaN** que puedo imputar gracias a la columna url, en esta se contiene el dato del 'id' y el de 'title'

*info() revela que el id esta mal registrado, tras la inputacion de los datos faltantes se debe convertir a datos INT*

In [5]:
pruebas = games[['url','id','title']]
id_nulo = pruebas[pruebas['id'].isna()]
print(id_nulo)
title_nulo = pruebas[pruebas['title'].isna()]
print('-----------------------------------------------------------------------------------------------------------')
print(title_nulo)


KeyError: "['url'] not in index"

In [6]:
print(games.loc[28700])
print(games.loc[2540])

genres          [Action, Open World, Batman, Adventure, Stealt...
title              Batman: Arkham City - Game of the Year Edition
id                                                         200260
release_year                                                 2012
Name: 28700, dtype: object
genres          [Action, Indie]
title                   Unknown
id                       317160
release_year               2014
Name: 2540, dtype: object


se asigna unknown para la fila ya que aun aporta valor para la columna "genres"

In [7]:
games.loc[2540,'title'] = 'Unknown'

In [8]:
def Imputar_Id(columna):
    indice = games[games[columna].isnull()].index
    for i in indice:
        url = games['url'][i]
        match = re.search(r"http://store.steampowered.com/app/(\d+)", url)
        
        if match:
            id = match.group(1)
            #debido al warning se hace cast() a la variable id para inputarlo con la columna 'id' dtype=float
            games.loc[indice,columna] = float(id)
        else:
            print("No se pudo extraer ID y TITLE de la URL.")

    

+ se utiliza la funcion para imputar el valor faltante y se convierte la columna a tipo INT
+ ya extraido el dato id , se elimina la columna url

In [9]:
Imputar_Id('id')
games['id'] = games['id'].astype(int)
games = games.drop('url',axis=1)
games.info()

KeyError: "['url'] not found in axis"

se borra las fechas de lanzamiento, ya no sera util para la primera funcion

In [10]:
games = games.drop('release_date',axis=1)
games.rename(columns={'id':'item_id'},inplace=True)
games.head()

KeyError: "['release_date'] not found in axis"

In [11]:
games.to_parquet('datos/games.parquet',index=False)